# Markdown to Mermaid Flowchart Converter

This notebook converts `.md` files containing Mermaid flowcharts to standalone `.mmd` files by:
1. Finding all Markdown files in a specified directory
2. Extracting the Mermaid flowchart content between triple backticks
3. Saving each flowchart as a separate `.mmd` file
4. Removing the triple backticks and the "mermaid" label

## 1. Import Required Libraries

In [ ]:
import os
import glob
import re
import shutil
from pathlib import Path

## 2. Set Up Directory Paths

In [ ]:
# Input directory containing .md files with Mermaid flowcharts
input_dir = os.path.join(os.getcwd(), "flowcharts")

# Output directory for .mmd files
output_dir = os.path.join(os.getcwd(), "mermaid_files")

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

print(f"Input directory: {input_dir}")
print(f"Output directory: {output_dir}")

## 3. Define Markdown to Mermaid Conversion Function

In [ ]:
def extract_mermaid_from_markdown(md_content):
    """
    Extract Mermaid flowchart from markdown content.
    
    Args:
        md_content (str): Markdown content with Mermaid flowchart
        
    Returns:
        str: Extracted Mermaid flowchart content or None if not found
    """
    # Pattern to match ```mermaid ... ``` blocks
    pattern = r'```mermaid\s*([\s\S]*?)```'
    
    # Find all matches
    matches = re.findall(pattern, md_content)
    
    if matches:
        # Return the first match (assuming one flowchart per file)
        return matches[0].strip()
    else:
        return None

## 4. Process Single File

In [ ]:
def process_file(md_file, output_dir):
    """
    Process a single markdown file and extract Mermaid content to .mmd file
    
    Args:
        md_file (str): Path to the markdown file
        output_dir (str): Directory to save the .mmd file
        
    Returns:
        tuple: (success, output_path or error message)
    """
    try:
        # Read markdown file
        with open(md_file, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Extract Mermaid content
        mermaid_content = extract_mermaid_from_markdown(content)
        
        if not mermaid_content:
            return (False, f"No Mermaid content found in {md_file}")
        
        # Create output filename with .mmd extension
        base_name = os.path.basename(md_file)
        file_name = os.path.splitext(base_name)[0]
        output_path = os.path.join(output_dir, f"{file_name}.mmd")
        
        # Write Mermaid content to .mmd file
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(mermaid_content)
        
        return (True, output_path)
        
    except Exception as e:
        return (False, f"Error processing {md_file}: {str(e)}")

## 5. Batch Process Directory

In [ ]:
def convert_directory(input_dir, output_dir):
    """
    Convert all markdown files in a directory to Mermaid files
    
    Args:
        input_dir (str): Directory containing .md files
        output_dir (str): Directory to save .mmd files
        
    Returns:
        dict: Statistics about the conversion process
    """
    # Find all .md files in the input directory
    md_files = glob.glob(os.path.join(input_dir, "*.md"))
    
    stats = {
        "total": len(md_files),
        "successful": 0,
        "failed": 0,
        "results": []
    }
    
    for md_file in md_files:
        print(f"Processing {md_file}...")
        success, result = process_file(md_file, output_dir)
        
        if success:
            stats["successful"] += 1
            status = "Success"
        else:
            stats["failed"] += 1
            status = "Failed"
            
        stats["results"].append({
            "file": md_file,
            "status": status,
            "result": result
        })
    
    return stats

## 6. Execute the Conversion

In [ ]:
# Execute the conversion process
stats = convert_directory(input_dir, output_dir)

# Print summary statistics
print("\n=== Conversion Summary ===")
print(f"Total files processed: {stats['total']}")
print(f"Successfully converted: {stats['successful']}")
print(f"Failed conversions: {stats['failed']}")

# Print detailed results
print("\n=== Detailed Results ===")
for result in stats['results']:
    print(f"{result['file']}: {result['status']} - {result['result']}")

## 7. Verify Output Files

In [ ]:
# Check and display the first few lines of each converted file
mmd_files = glob.glob(os.path.join(output_dir, "*.mmd"))

print(f"\nFound {len(mmd_files)} .mmd files in output directory")

# Preview the first few lines of each converted file (up to 5 files)
for i, mmd_file in enumerate(mmd_files[:5]):
    print(f"\n=== Contents of {os.path.basename(mmd_file)} ===")
    with open(mmd_file, 'r', encoding='utf-8') as f:
        # Read up to the first 10 lines
        lines = [next(f) for _ in range(10) if f]
        print('\n'.join(lines))
        print("...")
    
    if i >= 4 and len(mmd_files) > 5:
        print(f"\n(+ {len(mmd_files) - 5} more files)")
        break

## Conclusion

The notebook has successfully:
1. Identified all Markdown files in the input directory
2. Extracted Mermaid flowchart content by removing the triple backticks and "mermaid" markers
3. Saved the extracted content as standalone .mmd files in the output directory
4. Displayed a summary of the conversion process and previewed the output

You can now use these .mmd files with Mermaid rendering tools or services directly, without the Markdown wrapper.

## 8. Convert Mermaid Files to PNG Images

Now we'll convert all the generated `.mmd` files to `.png` images using the Kroki API.

In [ ]:
import requests
import base64
from tqdm.notebook import tqdm
import os
import json